In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
import pandas as pd
import numpy as np
from whitebox.eval import WhiteBoxSensitivity
import time
import datetime

In [2]:
df = pd.read_csv("sample_classifier_data.csv")

In [3]:
ydepend = 'response'
df['column1.bin'] = df['column1'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')
df['column2.bin'] = df['column2'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')

string_categories = df.select_dtypes(include = ['O'])
# iterate over string categories
for cat in string_categories:
    df[cat] = pd.Categorical(df[cat])
    
df.select_dtypes(include = ['category'])
model_df = df.copy(deep = True)


dummies = pd.concat([pd.get_dummies(model_df.loc[:, col], prefix = col) for col in model_df.select_dtypes(include = ['category']).columns], axis = 1)
finaldf = pd.concat([model_df.select_dtypes(include = [np.number]), dummies], axis = 1)

In [5]:


#clf = RandomForestClassifier()
#clf = SVC()
clf = BernoulliNB()
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])


WB = WhiteBoxSensitivity(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1.bin','column2.bin'],
                    aggregate_func=np.mean,
                    verbose=None,
                     std_num=2
                    )

    


In [6]:
starttime = time.time()
WB.run()
print "Run Time ",datetime.timedelta(seconds=time.time() - starttime)

Run Time  0:02:48.522000


In [7]:
WB.save('./test_svm_sensitivity.html')

In [8]:

from whitebox.eval import WhiteBoxError
WB = WhiteBoxError(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1.bin','column2.bin'],
                    aggregate_func=np.mean,
                    verbose=None
                    )

In [ ]:
starttime = time.time()
WB.run()
print "Run Time ",datetime.timedelta(seconds=time.time() - starttime)

In [ ]:
WB.save('./test_bernoullinb_error.html') 

In [6]:
pd.__version__

u'0.19.2'